In [1]:
from django.db import connection

from cinema.models import Person, Movie, MovieNomination, PersonNomination
from cinema.choices import MOVIE_GENRES
from cinema.utils import print_sql

In [2]:
from django.db import reset_queries

from cinema.populate_database import populate_database


populate_database()

# Reset queries so they won't impact future output
reset_queries()

## Filter

https://books.agiliq.com/projects/django-orm-cookbook/en/latest/join.html

In [9]:
queryset = Movie.objects.filter(genre=MOVIE_GENRES.action)
# str(queryset.query)
print_sql(queryset)

SELECT "cinema_movie"."id",
       "cinema_movie"."title",
       "cinema_movie"."sinopsis",
       "cinema_movie"."genre",
       "cinema_movie"."release_date",
       "cinema_movie"."available_on_netflix",
       "cinema_movie"."imdb_rate",
       "cinema_movie"."director_id"
FROM "cinema_movie"
WHERE "cinema_movie"."genre" = action



## Exclude

In [6]:
queryset = Movie.objects.exclude(genre__in=[MOVIE_GENRES.action, MOVIE_GENRES.mistery])
# str(queryset.query)
print_sql(queryset)

SELECT "cinema_movie"."id",
       "cinema_movie"."title",
       "cinema_movie"."sinopsis",
       "cinema_movie"."genre",
       "cinema_movie"."release_date",
       "cinema_movie"."available_on_netflix",
       "cinema_movie"."imdb_rate",
       "cinema_movie"."director_id"
FROM "cinema_movie"
WHERE NOT ("cinema_movie"."genre" IN (action,
                                      mistery))



## Filter: filtering across tables
https://docs.djangoproject.com/en/4.0/topics/db/queries/#lookups-that-span-relationships

In [14]:
queryset = Movie.objects.filter(director__name="Greta Gerwig").values_list("title", "director__name")
# str(queryset.query)
print_sql(queryset)

SELECT "cinema_movie"."title",
       "cinema_person"."name"
FROM "cinema_movie"
INNER JOIN "cinema_person" ON ("cinema_movie"."director_id" = "cinema_person"."id")
WHERE "cinema_person"."name" = Greta Gerwig



## select_related

In [3]:
reset_queries()

queryset = Movie.objects.all()

for movie in queryset:
    print(f"{movie.title} was directed by {movie.director.name}")

connection.queries

Deep Water was directed by Adrian Lyne


[{'sql': 'SELECT "cinema_movie"."id", "cinema_movie"."title", "cinema_movie"."sinopsis", "cinema_movie"."genre", "cinema_movie"."release_date", "cinema_movie"."available_on_netflix", "cinema_movie"."imdb_rate", "cinema_movie"."director_id" FROM "cinema_movie"',
  'time': '0.001'},
 {'sql': 'SELECT "cinema_person"."id", "cinema_person"."name", "cinema_person"."birth_date", "cinema_person"."death_date" FROM "cinema_person" WHERE "cinema_person"."id" = 30 LIMIT 21',
  'time': '0.001'}]

In [4]:
reset_queries()

queryset = Movie.objects.select_related("director").all()

for movie in queryset:
    print(f"{movie.title} was directed by {movie.director.name}")

connection.queries

Deep Water was directed by Adrian Lyne


[{'sql': 'SELECT "cinema_movie"."id", "cinema_movie"."title", "cinema_movie"."sinopsis", "cinema_movie"."genre", "cinema_movie"."release_date", "cinema_movie"."available_on_netflix", "cinema_movie"."imdb_rate", "cinema_movie"."director_id", "cinema_person"."id", "cinema_person"."name", "cinema_person"."birth_date", "cinema_person"."death_date" FROM "cinema_movie" LEFT OUTER JOIN "cinema_person" ON ("cinema_movie"."director_id" = "cinema_person"."id")',
  'time': '0.001'}]

In [5]:
reset_queries()

queryset = Movie.objects.select_related("director").only("title", "director").all()

for movie in queryset:
    print(f"{movie.title} was directed by {movie.director.name}")

connection.queries

Deep Water was directed by Adrian Lyne


[{'sql': 'SELECT "cinema_movie"."id", "cinema_movie"."title", "cinema_movie"."director_id", "cinema_person"."id", "cinema_person"."name", "cinema_person"."birth_date", "cinema_person"."death_date" FROM "cinema_movie" LEFT OUTER JOIN "cinema_person" ON ("cinema_movie"."director_id" = "cinema_person"."id")',
  'time': '0.001'}]

## prefetch_related

Works for FK relationships, but also for M2M relation ships. However, the optimization is made by Python and not the SQL (checkout .

In [6]:
reset_queries()

queryset = Movie.objects.all()

for movie in queryset:
    print(f"{movie.title} cast: \n")
    
    for person in movie.cast.all():
        print(f"- {person.name}\n")
        
connection.queries

Deep Water cast: 

- Ana de Armas

- Ben Affleck

- Jacob Elordi

- Finn Wittrock

- Rachel Blanchard



[{'sql': 'SELECT "cinema_movie"."id", "cinema_movie"."title", "cinema_movie"."sinopsis", "cinema_movie"."genre", "cinema_movie"."release_date", "cinema_movie"."available_on_netflix", "cinema_movie"."imdb_rate", "cinema_movie"."director_id" FROM "cinema_movie"',
  'time': '0.000'},
 {'sql': 'SELECT "cinema_person"."id", "cinema_person"."name", "cinema_person"."birth_date", "cinema_person"."death_date" FROM "cinema_person" INNER JOIN "cinema_movie_cast" ON ("cinema_person"."id" = "cinema_movie_cast"."person_id") WHERE "cinema_movie_cast"."movie_id" = 3',
  'time': '0.001'}]

In [7]:
reset_queries()

queryset = Movie.objects.prefetch_related("cast").all()

for movie in queryset:
    print(f"{movie.title} cast: \n")
    
    for person in movie.cast.all():
        print(f"- {person.name}\n")
        
connection.queries

Deep Water cast: 

- Ana de Armas

- Ben Affleck

- Jacob Elordi

- Finn Wittrock

- Rachel Blanchard



[{'sql': 'SELECT "cinema_movie"."id", "cinema_movie"."title", "cinema_movie"."sinopsis", "cinema_movie"."genre", "cinema_movie"."release_date", "cinema_movie"."available_on_netflix", "cinema_movie"."imdb_rate", "cinema_movie"."director_id" FROM "cinema_movie"',
  'time': '0.000'},
 {'sql': 'SELECT ("cinema_movie_cast"."movie_id") AS "_prefetch_related_val_movie_id", "cinema_person"."id", "cinema_person"."name", "cinema_person"."birth_date", "cinema_person"."death_date" FROM "cinema_person" INNER JOIN "cinema_movie_cast" ON ("cinema_person"."id" = "cinema_movie_cast"."person_id") WHERE "cinema_movie_cast"."movie_id" IN (3)',
  'time': '0.001'}]